In [1]:
%matplotlib inline
import imp
import utils
imp.reload(utils)
from utils import *
from __future__ import division, print_function

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Setup

In [2]:
path = "data/dogscats/"
model_path = path + 'models/'
if not os.path.exists(model_path): 
    os.mkdir(model_path)

batch_size=64

In [3]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In this notebook we're going to create an ensemble of models and use their average as our predictions. For each ensemble, we're going to follow our usual fine-tuning steps:

1) Create a model that retrains just the last layer

2) Add this to a model containing all VGG layers except the last layer

3) Fine-tune just the dense layers of this model (pre-computing the convolutional layers)

4) Add data augmentation, fine-tuning the dense layers without pre-computation.

So first, we need to create our VGG model and pre-compute the output of the conv layers:

In [5]:
model = Vgg16().model
conv_layers,fc_layers = split_at(model, Conv2D)

In [ ]:
conv_model = Sequential(conv_layers)

In [ ]:
val_features = conv_model.predict_generator(val_batches,verbose=1)

In [ ]:
val_features.shape

In [ ]:
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [ ]:
len(batches) # 训练集上有360步，360批次

In [ ]:
batches.n

In [ ]:
batches.next()[0].shape,batches.next()[1].shape

In [ ]:
batches_2 = get_data(path+'train')

In [ ]:
batches_2.shape

In [ ]:
# 因为显存不够只要分批次处理 训练集

batches_2_1 = batches_2[:10000]
batches_2_2 = batches_2[10000:20000]
batches_2_3 = batches_2[20000:]

In [ ]:
batches_2_1.shape,batches_2_2.shape,batches_2_3.shape

In [ ]:
trn_features_1 = conv_model.predict(batches_2_1,verbose=1)

In [ ]:
trn_features_2 = conv_model.predict(batches_2_2,verbose=1)

In [ ]:
trn_features_3 = conv_model.predict(batches_2_3,verbose=1)

In [ ]:
trn_features_1.shape,trn_features_2.shape,trn_features_3.shape

In [ ]:
# 存到硬盘里，还是内存太小，合并不了。

save_array(model_path + 'train_convlayer_features_1.bc', trn_features_1)
save_array(model_path + 'train_convlayer_features_2.bc', trn_features_2)
save_array(model_path + 'train_convlayer_features_3.bc', trn_features_3)

In [ ]:
trn_features = np.concatenate((trn_features_1,trn_features_2,trn_features_3))

In [ ]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)

In [ ]:
trn_features = conv_model.predict_generator(batches,verbose=1) # 预计算卷积层的输出

In [ ]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)

In [ ]:
trn_features.shape

In the future we can just load these precomputed features:

In [ ]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

We can also save some time by pre-computing the training and validation arrays with the image decoding and resizing already done:

In [6]:
trn = get_data(path+'train')

Found 23000 images belonging to 2 classes.


In [7]:
save_array(model_path+'train_data.bc', trn)

In [8]:
val = get_data(path+'valid')

Found 2000 images belonging to 2 classes.


In [9]:
save_array(model_path+'valid_data.bc', val)

In the future we can just load these resized images:

In [ ]:
trn = load_array(model_path+'train_data.bc')
val = load_array(model_path+'valid_data.bc')

Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [11]:
model.pop()
model.pop()

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [13]:
ll_val_feat = model.predict_generator(val_batches)

In [14]:
ll_feat = model.predict_generator(batches)

In [15]:
save_array(model_path + 'train_ll_feat.bc', ll_feat)
save_array(model_path + 'valid_ll_feat.bc', ll_val_feat)

In [ ]:
ll_feat = load_array(model_path+ 'train_ll_feat.bc')
ll_val_feat = load_array(model_path + 'valid_ll_feat.bc')

...and let's also grab the test data, for when we need to submit:

In [18]:
test = get_data(path+'test')
save_array(model_path+'test_data.bc', test)

Found 12500 images belonging to 1 classes.


In [ ]:
test = load_array(model_path+'test_data.bc')

## Last layer

The functions automate creating a model that trains the last layer from scratch, and then adds those new layers on to the main model.

In [19]:
def get_ll_layers():
    return [ 
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax') 
        ]

In [29]:
def train_last_layer(i):
    ll_layers = get_ll_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr=1e-5
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), epochs=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), epochs=1)
    ll_model.save_weights(model_path+'ll_bn' + i + '.h5')

    vgg = Vgg16()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    ll_layers = get_ll_layers()
    for layer in ll_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(model_path+'bn' + i + '.h5')
    return model

## Dense model

In [21]:
def get_conv_model(model):
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                         if type(layer) is Convolution2D][-1]

    conv_layers = layers[:last_conv_idx+1]
    conv_model = Sequential(conv_layers)
    fc_layers = layers[last_conv_idx+1:]
    return conv_model, fc_layers, last_conv_idx

In [22]:
def get_fc_layers(p, in_shape):
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ]

In [27]:
def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    fc_model.fit(trn_features, trn_labels, epochs=2, 
         batch_size=batch_size, validation_data=(val_features, val_labels))

    gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, 
       width_zoom_range=0.05, zoom_range=0.05,
       channel_shift_range=10, height_shift_range=0.05, shear_range=0.05, horizontal_flip=True)
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, 
                      shuffle=False, batch_size=batch_size)

    for layer in conv_model.layers: 
        layer.trainable = False
    for layer in get_fc_layers(0.5, conv_shape): 
        conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    conv_model.save_weights(model_path+'no_dropout_bn' + i + '.h5')
    conv_model.fit_generator(batches, epochs=1, 
                            validation_data=val_batches)
    for layer in conv_model.layers[16:]: 
        layer.trainable = True
    conv_model.fit_generator(batches, epochs=8, 
                            validation_data=val_batches)

    conv_model.optimizer.lr = 1e-7
    conv_model.fit_generator(batches, epochs=10, 
                            validation_data=val_batches)
    conv_model.save_weights(model_path + 'aug' + i + '.h5')

## Build ensemble

In [30]:
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)

Train on 23000 samples, validate on 2000 samples
Epoch 1/12
23000/23000 [==============================] - 3s 132us/step - loss: 0.7336 - acc: 0.7085 - val_loss: 0.4151 - val_acc: 0.8225
Epoch 2/12
23000/23000 [==============================] - 3s 113us/step - loss: 0.5241 - acc: 0.7948 - val_loss: 0.3434 - val_acc: 0.8595
Epoch 3/12
23000/23000 [==============================] - 3s 112us/step - loss: 0.4644 - acc: 0.8248 - val_loss: 0.3068 - val_acc: 0.8760
Epoch 4/12
23000/23000 [==============================] - 3s 114us/step - loss: 0.4290 - acc: 0.8403 - val_loss: 0.2847 - val_acc: 0.8885
Epoch 5/12
23000/23000 [==============================] - 3s 113us/step - loss: 0.3931 - acc: 0.8518 - val_loss: 0.2677 - val_acc: 0.8930
Epoch 6/12
23000/23000 [==============================] - 3s 114us/step - loss: 0.3749 - acc: 0.8591 - val_loss: 0.2555 - val_acc: 0.8975
Epoch 7/12
23000/23000 [==============================] - 3s 115us/step - loss: 0.3713 - acc: 0.8648 - val_loss: 0.2453 - v

ValueError: You called `set_weights(weights)` on layer "batch_normalization_9" with a  weight list of length 0, but the layer was expecting 4 weights. Provided weights: []...

## Combine ensemble and test

In [ ]:
ens_model = vgg_ft(2)
for layer in ens_model.layers: layer.trainable=True

In [ ]:
def get_ens_pred(arr, fname):
    ens_pred = []
    for i in range(5):
        i = str(i)
        ens_model.load_weights('{}{}{}.h5'.format(model_path, fname, i))
        preds = ens_model.predict(arr, batch_size=batch_size)
        ens_pred.append(preds)
    return ens_pred

In [ ]:
val_pred2 = get_ens_pred(val, 'aug')

In [ ]:
val_avg_preds2 = np.stack(val_pred2).mean(axis=0)

In [ ]:
categorical_accuracy(val_labels, val_avg_preds2).eval()